In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from statsmodels.tsa.arima_model import ARIMA

from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [44]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [45]:
path = "../data/parquet/dataSetFinalTratadoTop5Diseases.parquet"
df = pd.read_parquet(path)

### Funções Úteis

In [46]:
def verificar_periodo_completo(dataframe, coluna):
    data_mais_antiga = dataframe[coluna].min()
    data_mais_recente = dataframe[coluna].max()
    
    intervalo_completo = pd.period_range(start=data_mais_antiga, end=data_mais_recente, freq='M')
    
    todas_datas_presentes = all(data in dataframe[coluna].values for data in intervalo_completo)

    return data_mais_antiga, data_mais_recente, todas_datas_presentes

In [47]:
def recortar_dataframe(dataframe, coluna, data_inicio, data_fim):
    data_inicio = pd.Period(data_inicio, freq='M')
    data_fim = pd.Period(data_fim, freq='M')
    
    dataframe_recortado = dataframe[(dataframe[coluna] >= data_inicio) & (dataframe[coluna] <= data_fim)]
    
    return dataframe_recortado

In [48]:
def concatenar_datasets(*dataframes):
    dataframe_concatenado = pd.concat(dataframes, ignore_index=True)
    return dataframe_concatenado

In [49]:
def prever_arima(serie_temporal):
    teste_serie_regiao = serie_temporal.groupby('unique_id').tail(12)
    train_serie_regiao = serie_temporal.drop(teste_serie_regiao.index, axis = 0)

    models = [AutoARIMA(season_length = 12)]

    sf = StatsForecast(
    df=train_serie_regiao,
    models=models,
    freq='M'
    )

    df_forecast = sf.forecast(12, fitted=True).reset_index()

    df_fitted = sf.forecast_fitted_values().reset_index()

    return df_forecast, df_fitted

In [50]:
def modelar_residuos(serie_temporal, forecast, fitted):
    residuos = pd.DataFrame()

    for id in forecast['unique_id'].unique():
        treino = fitted[fitted['unique_id'] == id]
        teste = forecast[forecast['unique_id'] == id].reset_index(drop=True)

        residuos_treino = (treino['AutoARIMA'] - treino['y'])

        teste_serie = serie_temporal[serie_temporal['unique_id'] == id].tail(12).reset_index(drop=True)

        teste['ds'] = teste['ds'] + pd.DateOffset(months=1)

        residuos_teste = (forecast['AutoARIMA'] - teste_serie['y'])
        residuos_totais = pd.concat([residuos_treino, residuos_teste], ignore_index=True)

        coluna_nome = f'residuo_{id}'
        residuos[coluna_nome] = residuos_totais

    return residuos

In [51]:
def criar_lags(data, lag):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [52]:
def prever_mlp(residuos, window_size):
    df_prev = pd.DataFrame()

    for id in residuos.columns:
        res = residuos[id].values

        X, y = criar_lags(res, window_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        mlp = MLPRegressor(hidden_layer_sizes=(100,100), max_iter=500, random_state=42)
        mlp.fit(X_train, y_train)
        y_pred = mlp.predict(X_test)

        prev_full = np.concatenate((mlp.predict(X_train), y_pred), axis=0)

        coluna_nome = f'prev_{id}'
        df_prev[coluna_nome] = prev_full

    return df_prev

In [53]:
def concatenar_datasets(*dataframes):
    dataframe_concatenado = pd.concat(dataframes, ignore_index=True)
    return dataframe_concatenado

### Série Por Região (Sem distinguir doenças)

In [54]:
def pegar_dfs_por_regiao(df_full):
    df_regioes = df_full[['Data de Diagnostico', 'Regiao', 'categoria_doenca']]
    df_regioes['y'] = 1

    df_regioes.dropna(inplace=True)

    df_regioes['year_month'] = df_regioes['Data de Diagnostico'].dt.to_period('M')

    df_grouped = df_regioes.groupby(['year_month', 'Regiao', 'categoria_doenca'])['y'].sum().reset_index()

    df_nordeste = df_grouped[df_grouped['Regiao'] == 'NORDESTE']
    df_sudeste = df_grouped[df_grouped['Regiao'] == 'SUDESTE']
    df_norte = df_grouped[df_grouped['Regiao'] == 'NORTE']
    df_centro_oeste = df_grouped[df_grouped['Regiao'] == 'CENTRO-OESTE']
    df_sul = df_grouped[df_grouped['Regiao'] == 'SUL']

    dfs = [df_nordeste, df_sudeste, df_norte, df_centro_oeste, df_sul]

    serie_nordeste = pd.DataFrame()
    serie_sudeste = pd.DataFrame()
    serie_norte = pd.DataFrame()
    serie_centro_oeste = pd.DataFrame()
    serie_sul = pd.DataFrame()

    for df in dfs:
        mama = df[df['categoria_doenca'] == 'Câncer de Mama']
        prostata = df[df['categoria_doenca'] == 'Câncer de Próstata']
        utero = df[df['categoria_doenca'] == 'Câncer de Colo do Útero']
        pele = df[df['categoria_doenca'] == 'Câncer de Pele']
        pulmao = df[df['categoria_doenca'] == 'Câncer de Pulmão']
        outros = df[df['categoria_doenca'] == 'Outros']

        mama = mama.drop('Regiao', axis=1)
        prostata = prostata.drop('Regiao', axis=1)
        utero = utero.drop('Regiao', axis=1)
        pele = pele.drop('Regiao', axis=1)
        pulmao = pulmao.drop('Regiao', axis=1)
        outros = outros.drop('Regiao', axis=1)

        serie = concatenar_datasets(mama, prostata, utero, pele, pulmao, outros)
        serie.rename(columns={'year_month': 'ds', 'categoria_doenca': 'unique_id'}, inplace=True)
        serie['ds'] = serie['ds'].apply(lambda x: x.to_timestamp())

        if df.equals(df_nordeste):
            serie_nordeste = serie
        elif df.equals(df_sudeste):
            serie_sudeste = serie
        elif df.equals(df_norte):
            serie_norte = serie
        elif df.equals(df_centro_oeste):
            serie_centro_oeste = serie
        elif df.equals(df_sul):
            serie_sul = serie

    return serie_nordeste, serie_sudeste, serie_norte, serie_centro_oeste, serie_sul

    # df_nordeste = recortar_dataframe(df_nordeste, 'year_month', '1999-01', '2018-12')
    # df_sudeste = recortar_dataframe(df_sudeste, 'year_month', '1999-01', '2018-12')
    # df_norte = recortar_dataframe(df_norte, 'year_month', '1999-01', '2018-12')
    # df_centro_oeste = recortar_dataframe(df_centro_oeste, 'year_month', '1999-01', '2018-12')
    # df_sul = recortar_dataframe(df_sul, 'year_month', '1999-01', '2018-12')

In [55]:
serie_nordeste, serie_sudeste, serie_norte, serie_centro_oeste, serie_sul = pegar_dfs_por_regiao(df)
serie_nordeste.head(20)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12176\2896319255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regioes['y'] = 1
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12176\2896319255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regioes.dropna(inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12176\2896319255.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,ds,unique_id,y
0,1990-01-01,Câncer de Mama,19
1,1990-02-01,Câncer de Mama,14
2,1990-03-01,Câncer de Mama,24
3,1990-04-01,Câncer de Mama,24
4,1990-05-01,Câncer de Mama,27
5,1990-06-01,Câncer de Mama,26
6,1990-07-01,Câncer de Mama,125
7,1990-08-01,Câncer de Mama,34
8,1990-09-01,Câncer de Mama,27
9,1990-10-01,Câncer de Mama,26


In [56]:
forecast, fitted = prever_arima(serie_nordeste)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\statsforecast\core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\statsforecast\core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.

In [57]:
serie_residuos = modelar_residuos(serie_nordeste, forecast, fitted)

In [59]:
serie_residuos.head()

,residuo_Câncer de Colo do Útero,residuo_Câncer de Mama,residuo_Câncer de Pele,residuo_Câncer de Próstata,residuo_Câncer de Pulmão,residuo_Outros
0,-0.020000,-0.018999,-0.033001,-0.008000,-0.005000,-0.067001
1,1.562130,4.040979,-5.578915,-0.000004,-3.212999,-16.393059
2,-5.399590,-7.016167,-10.829559,-0.000002,-0.700732,-15.048958
3,-24.174761,-4.032589,4.418842,3.803312,-1.324407,11.738075
4,2.580128,-5.246496,2.327557,-0.750585,-2.872223,3.741043


In [60]:
forecast.head()

,unique_id,ds,AutoARIMA
0,Câncer de Colo do Útero,2017-12-31,23.970091
1,Câncer de Colo do Útero,2018-01-31,19.008041
2,Câncer de Colo do Útero,2018-02-28,19.023897
3,Câncer de Colo do Útero,2018-03-31,19.578400
4,Câncer de Colo do Útero,2018-04-30,21.450970


In [61]:
fitted.head()

,unique_id,ds,y,AutoARIMA
0,Câncer de Colo do Útero,1990-01-01,20.0,19.980000
1,Câncer de Colo do Útero,1990-02-01,18.0,19.562130
2,Câncer de Colo do Útero,1990-03-01,25.0,19.600410
3,Câncer de Colo do Útero,1990-04-01,47.0,22.825239
4,Câncer de Colo do Útero,1990-05-01,27.0,29.580128


### Série Por Região e Categoria de Doença